50 クラス属性に__set_name__で注釈を加える

In [2]:
class Field:
    def __init__(self, name):
        self.name = name
        self.internal_name = "_" + self.name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name,  '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

In [6]:
class Customer:
    first_name = Field('first_name')
    last_name = Field('last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')

In [8]:
cust = Customer()
print(f'Before:{cust.first_name!r} {cust.__dict__}')
cust.first_name = "Euclid"
print(f'After:{cust.first_name!r} {cust.__dict__}')
#クラス定義が冗長

Before:'' {}
After:'Euclid' {'_first_name': 'Euclid'}


In [17]:
#冗長性をなくすため、メタクラスを使う
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

In [18]:
class DatabaseRow(object, metaclass = Meta):
    pass

class Field:
    def __init__(self):
        #ここはメタクラスで代入
        self.name = None
        self.internal_name = None

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)



In [19]:
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

In [20]:
cust = BetterCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Euler'
print(f'After; {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After; 'Euler' {'_first_name': 'Euler'}


In [29]:
#__set_name__を使う
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None

    def __set_name__(self, owner, name):
        self.name = name
        self.internal_name = '_' + name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

In [30]:
class FixedCustomer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

In [31]:
cust = FixedCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Mersenne'
print(f'After:  {cust.first_name!r} {cust.__dict__}')

Before: '' {}
After:  'Mersenne' {'_first_name': 'Mersenne'}
